In [1]:
import sys
from elasticsearch.helpers import BulkIndexError
import json
from elasticsearch import helpers,Elasticsearch
from dotenv import load_dotenv
import os

In [45]:


load_dotenv()

client = Elasticsearch(
    ["https://localhost:9200"],  # Địa chỉ Elasticsearch
    basic_auth=("elastic", os.getenv('ELASTIC_PASSWORD')),  # Thay your_password bằng ELASTIC_PASSWORD
    # verify_certs=False  # Bỏ kiểm tra SSL (hoặc cung cấp đường dẫn CA nếu cần)
    ca_certs="ca.crt",
    request_timeout=60

)


In [46]:
client.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KH-rAW4aTLSEJm4gLMx3mg', 'version': {'number': '8.17.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c', 'build_date': '2025-02-05T22:10:57.067596412Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [29]:
client.ping()

True

In [38]:

resp = client.indices.put_index_template(
    name="bgl-template",
    index_patterns=[
        "logs-bgl-*"
    ],
    data_stream={},
    template={
        "settings": {
            "index.mode": "logsdb",
            "number_of_shards": 1,  # Giảm số shard nếu log không quá lớn
        },
        "mappings": {
            "properties": {
                "Label": { "type": "keyword" },
                "@Timestamp": { "type": "long" },
                "Date": { "type": "keyword" },
                "Node": { "type": "keyword" },
                "Time": { "type": "date", "format": "yyyy-MM-dd-HH.mm.ss.SSSSSS" },
                "NodeRepeat": { "type": "keyword" },
                "Type": { "type": "keyword" },
                "Component": { "type": "keyword" },
                "Level": { "type": "keyword" },
                "Content": { "type": "text" }  # Hỗ trợ tìm kiếm lỗi
            }
        }
    },
    priority=101,
)
print(resp)

{'acknowledged': True}


In [12]:

# Đọc file log
file_path = "data/BGL/BGL_2k.log"

with open(file_path, "r", encoding="utf-8") as file:

    for line in file:

        fields  = line.strip().split()  # Tách theo khoảng trắng

        if len(fields) < 10:
            continue  # Bỏ qua dòng không đủ dữ liệu

        log_entry = {
            "Label": fields[0],
            "@Timestamp": int(fields[1]),
            "Date": fields[2],
            "Node": fields[3],
            "Time": fields[4],
            "NodeRepeat": fields[5],
            "Type": fields[6],
            "Component": fields[7],
            "Level": fields[8],
            "Content": " ".join(fields[9:])  # Nội dung log có thể chứa dấu phẩy
        }

        # Ghi vào Elasticsearch
        client.index(index="logs-bgl-2025.03.10", document=log_entry)
        # print(log_entry)

print("Đẩy log vào Elasticsearch thành công!")

Đẩy log vào Elasticsearch thành công!


In [4]:
query = {
    "query": {
        "match": {
            "Level": "INFO"
        }
    }
}

# Gửi yêu cầu tìm kiếm
print(client.search(index="logs-bgl-*", body=query))

{'took': 99, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1597, 'relation': 'eq'}, 'max_score': 0.22520715, 'hits': [{'_index': '.ds-logs-bgl-2025.03.10-2025.03.09-000001', '_id': 'AZV76oVAXbZOegpCHfXT', '_score': 0.22520715, '_source': {'Label': '-', '@timestamp': 1117838570, 'Date': '2005.06.03', 'Node': 'R02-M1-N0-C:J12-U11', 'Time': '2005-06-03-15.42.50.675872', 'NodeRepeat': 'R02-M1-N0-C:J12-U11', 'Type': 'RAS', 'Component': 'KERNEL', 'Level': 'INFO', 'Content': 'instruction cache parity error corrected'}}, {'_index': '.ds-logs-bgl-2025.03.10-2025.03.09-000001', '_id': 'AZV76oVAXbZOegpDHUXU', '_score': 0.22520715, '_source': {'Label': '-', '@timestamp': 1117838573, 'Date': '2005.06.03', 'Node': 'R02-M1-N0-C:J12-U11', 'Time': '2005-06-03-15.42.53.276129', 'NodeRepeat': 'R02-M1-N0-C:J12-U11', 'Type': 'RAS', 'Component': 'KERNEL', 'Level': 'INFO', 'Content': 'instruction cache parity error corrected'}}, {'_index'

In [48]:

# Đọc file log
file_path_BGL = "data/BGL/BGL_2k.log"
batch_size = 1000
actions = []

with open(file_path_BGL, "r", encoding="utf-8") as file:

    for line in file:

        fields  = line.strip().split()  # Tách theo khoảng trắng

        if len(fields) < 10:
            continue  # Bỏ qua dòng không đủ dữ liệu

        log_entry = {
            "_index": "logs-bgl-stream",  # Tên phải khớp với index_patterns "logs-bgl-*"
            "_op_type": "create",
            "_source": {
                "Label": fields[0],
                "@timestamp": int(fields[1]),
                "Date": fields[2],
                "Node": fields[3],
                "Time": fields[4],
                "NodeRepeat": fields[5],
                "Type": fields[6],
                "Component": fields[7],
                "Level": fields[8],
                "Content": " ".join(fields[9:])  # Nội dung log có thể chứa dấu phẩy
            }
        }
        # print(log_entry)
        #
        # break
        actions.append(log_entry)

        # Gửi dữ liệu sau mỗi batch_size log
        if actions:
            try:
                helpers.bulk(client, actions)
            except BulkIndexError as e:
                print("Lỗi khi đẩy dữ liệu:")
                for error in e.errors[:5]:  # In 5 lỗi đầu tiên
                    print(json.dumps(error, indent=2))
                print(f"Đã dừng quá trình do lỗi trên. Tổng số lỗi: {len(e.errors)}")
                sys.exit(1)  # Thoát chương trình với mã lỗi 1

    # Gửi phần còn lại nếu còn log
    if actions:
        try:
            helpers.bulk(client, actions)
        except BulkIndexError as e:
            print("Lỗi khi đẩy dữ liệu:")
            for error in e.errors[:5]:  # In 5 lỗi đầu tiên
                print(json.dumps(error, indent=2))
            print(f"Đã dừng quá trình do lỗi trên. Tổng số lỗi: {len(e.errors)}")
            sys.exit(1)  # Thoát chương trình với mã lỗi 1


print("Đẩy log vào Elasticsearch thành công!")

Đẩy log vào Elasticsearch thành công!


In [49]:


query = {
    "query": {
        "match_all": {}
    }
}

# Lấy toàn bộ dữ liệu với helpers.scan() để tránh giới hạn `size`
logs = []
for hit in helpers.scan(client, index="logs-bgl-stream", query=query):
    logs.append(hit["_source"])  # Lấy toàn bộ _source

# Ghi dữ liệu vào file JSON
with open("data/BGL/BGL_2k.json", "w", encoding="utf-8") as f:
    json.dump(logs, f, indent=4, ensure_ascii=False)

print(f"✅ Xuất {len(logs)} log vào logs_output.json thành công!")

✅ Xuất 2003000 log vào logs_output.json thành công!


In [50]:


# Read the JSON file
with open('data/BGL/BGL_2k.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Open a new file for writing in JSON Lines format
with open('data/BGL/BGL_2k.jsonl', 'w', encoding='utf-8') as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item) + '\n')

print("Conversion to JSON Lines format completed successfully!")


Conversion to JSON Lines format completed successfully!


In [4]:
import psycopg2



# Kết nối PostgreSQL
conn = psycopg2.connect(
    dbname="detection-db",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5433"
)
cursor = conn.cursor()
# Kiểm tra kết nối
cursor.execute("SELECT version();")
print("Connected to:", cursor.fetchone()[0])

# Đọc file JSON theo dòng và chèn vào PostgreSQL
with open("data/BGL/BGL_2k.jsonl", "r") as f:
    for line in f:

        record = json.loads(line.strip())  # Chuyển mỗi dòng JSON thành dict
        query = "INSERT INTO logs_bgl (logs_data) VALUES (%s);"
        cursor.execute(query, (json.dumps(record),))  # Chuyển dict thành JSON string
# Commit và đóng kết nối
conn.commit()
cursor.close()
conn.close()

Connected to: PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [ ]:
# client.indices.delete(index="logs-bgl-full")